<a href="https://colab.research.google.com/github/urvashi2004/PlantDoctor/blob/main/Colab/PlantDoktor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import os

from google.colab import drive
drive.mount('/content/drive')

# Define paths to your dataset
train_data_dir = '/content/drive/My Drive/Colab Notebooks/PlantDoctor/PlantVillageDS'
output_dir = '/content/drive/My Drive/Colab Notebooks/PlantDoctor'

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Define the model using MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the checkpoint callback
checkpoint_path = os.path.join(output_dir, 'model_checkpoint.keras')  # Changed extension to .keras
checkpoint = ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, monitor='val_loss')

# Train the model and save the training history
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,  # Updated batch size
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32,  # Updated batch size
    callbacks=[checkpoint]
)

# Save the entire model
model_save_path = os.path.join(output_dir, 'PlantDiseaseModel2.h5')
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

# Save training history to CSV
history_df = pd.DataFrame(history.history)
history_csv_path = os.path.join(output_dir, 'TrainingHistory2.csv')
history_df.to_csv(history_csv_path, index=False)
print(f"Training history saved to {history_csv_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 16511 images belonging to 3 classes.
Found 4127 images belonging to 3 classes.


<ipython-input-20-6cf98df24d83>:43: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


Epoch 1/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 290s 424ms/step - accuracy: 0.9322 - loss: 0.1954 - val_accuracy: 0.8794 - val_loss: 0.7427
Epoch 2/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0579 - val_accuracy: 0.7742 - val_loss: 1.4718
Epoch 3/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 196s 376ms/step - accuracy: 0.9779 - loss: 0.0740 - val_accuracy: 0.8411 - val_loss: 1.6687
Epoch 4/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 0s 379us/step - accuracy: 1.0000 - loss: 0.0121 - val_accuracy: 0.8065 - val_loss: 1.6810
Epoch 5/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 197s 378ms/step - accuracy: 0.9862 - loss: 0.0475 - val_accuracy: 0.8650 - val_loss: 1.2584
Epoch 6/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 0s 436us/step - accuracy: 1.0000 - loss: 0.0167 - val_accuracy: 0.7742 - val_loss: 2.1407
Epoch 7/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 199s 382ms/step - accuracy: 0.9876 - loss: 0.0352 - val_accuracy: 0.8020 - val_loss: 3.1537
Epoch 8/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 0s 405us/step - accuracy: 0.9688 - loss: 0.0

Model saved to /content/drive/My Drive/Colab Notebooks/PlantDoctor/PlantDiseaseModel2.h5
Training history saved to /content/drive/My Drive/Colab Notebooks/PlantDoctor/TrainingHistory2.csv


In [ ]:
from tensorflow.keras.models import load_model

# Load the model with custom objects if needed
model = load_model(model_path, custom_objects={})


ValueError: Layer "dense_18" expects 1 input(s), but it received 2 input tensors. Inputs received: [<KerasTensor shape=(None, 5, 5, 1280), dtype=float32, sparse=False, name=keras_tensor_4114>, <KerasTensor shape=(None, 5, 5, 1280), dtype=float32, sparse=False, name=keras_tensor_4115>]

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files
import tensorflow as tf

# Function to predict plant disease
def predict_plant_disease(model, img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize

    predictions = model.predict(img_array)
    class_indices = {v: k for k, v in train_generator.class_indices.items()}  # Reverse mapping
    predicted_class = np.argmax(predictions[0])
    class_name = class_indices[predicted_class]

    return class_name

# Upload the image
uploaded = files.upload()

# Get the path of the uploaded file
img_path = list(uploaded.keys())[0]
print(f"Image path: {img_path}")

# Load the model (if not already loaded)
model_path = '/content/drive/My Drive/Colab Notebooks/PlantDoctor/PlantDiseaseModel2.h5'
model = tf.keras.models.load_model(model_path)

# Predict the plant disease
predicted_class = predict_plant_disease(model, img_path)
print(f"The predicted class is: {predicted_class}")